# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

import hvplot as hv

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pprint import pprint
from datetime import datetime

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,-10.09,67,75,8.23,SJ,1732976643
1,1,tikhoretsk,45.8547,40.1253,2.51,67,22,3.41,RU,1732976644
2,2,afaahiti,-17.7500,-149.2833,25.57,84,100,6.29,PF,1732976645
3,3,tiksi,71.6872,128.8694,-19.90,98,100,2.92,RU,1732976646
4,4,shimoda,34.6667,138.9500,10.48,74,0,8.95,JP,1732976647


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
cities_worldmap = city_data_df.hvplot.points("Lng", 
                                            "Lat", 
                                            geo = True, 
                                            size = "Humidity",
                                            scale = 1,
                                            color = "City",
                                            alpha = 0.5,
                                            tiles = "OSM",
                                            frame_width = 1000,
                                            frame_height = 600
                                            )

# Display the map
cities_worldmap

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & \
                                    (city_data_df["Max Temp"] > 23) & \
                                    (city_data_df["Wind Speed"] < 4.5) &\
                                    (city_data_df["Cloudiness"] == 0),:
                                    ]


# Drop any rows with null values
print(f"Count of rows with null values:{filtered_city_df.isnull().sum(axis=1).sum()}")
print("If we had any Null values we could have dropped them using - filtered_city_df=filtered_city_df.dropna()" )

# Display sample data
filtered_city_df=filtered_city_df.sort_values(by=['Max Temp',"Humidity"], ascending=False).reset_index()
filtered_city_df

Count of rows with null values:0
If we had any Null values we could have dropped them using - filtered_city_df=filtered_city_df.dropna()


,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,113,113,manzini,-26.2500,31.5000,28.19,33,0,3.70,SZ,1732976772
1,292,292,gebeit,21.0667,36.3167,25.44,29,0,1.67,SD,1732976992
2,539,539,sattahip,12.6664,100.9007,25.05,78,0,3.60,TH,1732977290
3,479,479,tazacorte,28.6290,-17.9293,23.54,69,0,2.06,ES,1732977218
4,301,301,arica,-18.4750,-70.3042,23.13,53,0,4.12,CL,1732977002


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_df[["City", "Country", "Lat", "Lng", 'Max Temp',"Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Hotel Name
0,manzini,SZ,-26.2500,31.5000,28.19,33,
1,gebeit,SD,21.0667,36.3167,25.44,29,
2,sattahip,TH,12.6664,100.9007,25.05,78,
3,tazacorte,ES,28.6290,-17.9293,23.54,69,
4,arica,CL,-18.4750,-70.3042,23.13,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 5000
params = {}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{row['Lng']},{row['Lat']},{radius}"
    params["bias"] = f"proximity:{row['Lng']},{row['Lat']}"
    params["limit"] = 2
    params["apiKey"] = geoapify_key
    params["categories"] = "accommodation.hotel"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    with requests.get(url=base_url, params=params) as response:

    # Make and API request using the params dictionary and get API response to JSON format
        name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
manzini - nearest hotel: No hotel found
gebeit - nearest hotel: No hotel found
sattahip - nearest hotel: blue warehouse home
tazacorte - nearest hotel: App Leyma
arica - nearest hotel: Hotel Andalucía 


,City,Country,Lat,Lng,Max Temp,Humidity,Hotel Name
0,manzini,SZ,-26.2500,31.5000,28.19,33,No hotel found
1,gebeit,SD,21.0667,36.3167,25.44,29,No hotel found
2,sattahip,TH,12.6664,100.9007,25.05,78,blue warehouse home
3,tazacorte,ES,28.6290,-17.9293,23.54,69,App Leyma
4,arica,CL,-18.4750,-70.3042,23.13,53,Hotel Andalucía


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points("Lng", 
                                "Lat", 
                                geo = True, 
                                size = "Humidity",
                                scale = 1,
                                color = "City",
                                alpha = 1,
                                tiles = "OSM",
                                frame_width = 1000,
                                frame_height = 600,
                                hover_cols = ["Hotel Name", "Country"]
                                )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)